# Revised case normalization for Hirslanden Beau-Site 2017

Notes:

- There is no data for the hospital 'Hirslanden Beau Site' in 2017 in the DB.


In [1]:
import sys
sys.path.insert(0, '/home/jovyan/work')

from src.revised_case_normalization.notebook_functions.global_configs import *
from src.revised_case_normalization.notebook_functions.normalize import normalize
from src.revised_case_normalization.notebook_functions.revise import revise
from src.revised_case_normalization.notebook_functions.group import group
from src.revised_case_normalization.notebook_functions.update_db import update_db

In [2]:
file_info = FileInfo(os.path.join(ROOT_DIR, 'raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx'),
                     'Hirslanden Klinik Beau-Site', '2017', 'Änderungen Beau Site 2017')
print(file_info)


FileInfo(path='/home/jovyan/work/src/revised_case_normalization/raw_data/HI-Bern_Salem_Beau Site_Linde.xlsx', hospital_name_db='Hirslanden Klinik Beau-Site', year='2017', sheet='Änderungen Beau Site 2017')


In [3]:
revised_cases_df = normalize(file_info)
revised_cases_df

2022-11-21 08:56:36.797 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:38 - Read 47 cases for Hirslanden Klinik Beau-Site 2017
2022-11-21 08:56:36.801 | INFO     | src.revised_case_normalization.notebook_functions.normalize:normalize:73 - TYPES:
datum                             string
tranche                           string
case_id                           object
patient_id                        object
gender                            string
age_years                          int64
bfs_code                          string
duration_of_stay                   int64
pflegetage neu                    string
pccl                               int64
pccl neu                          string
old_pd                            string
primary_diagnosis                 string
added_icds                        string
removed_icds                      string
added_chops                       string
removed_chops                     string
drg                  

,case_id,patient_id,gender,age_years,duration_of_stay,case_id_norm,old_pd,primary_diagnosis,bfs_code,pccl,drg,added_icds,removed_icds,added_chops,removed_chops
0,41225861,D9798CB1880ECA77,M,63,5,41225861,C180,C180,M200,3,G10B,[C786],[],[],[]
1,41246909,1CF41AC1C2D10D62,M,83,3,41246909,J22,J22,M100,3,E75C,[D706],[],[],[]
2,41250682,89A046670053E7A4,W,76,17,41250682,K9180,K9180,M100,3,G70B,"[D630, D706]",[],[],[]
3,41256307,F14F340A6A8A29A5,W,77,5,41256307,N390,N390,M100,3,L63D,[K743],[],[],[]
4,41258314,35B85BE2EFA7811C,W,81,9,41258314,C56,C56,M100,3,N60B,[A099],[],[],[]
5,41272582,32E9366FA2B87CBA,M,55,4,41272582,K922,K922,M100,3,G48B,"[K500, K703, E871]",[K521],[],[]
6,41279857,150338288151836,M,64,9,41279857,I210,I210,M100,3,F41B,"[I1100, I313, I513]",[I1090],[],[]
7,41300053,EC6D4A9D2B502F76,M,66,6,41300053,C61,C61,M200,3,M01B,[C775],[],[],[]
8,41301906,EA46555953051562,W,81,5,41301906,K625,K625,M100,3,G48B,"[D500, I441]",[],[],[]
9,41269269,0BE197D58B3DE27B,W,72,15,41269269,I7029,I7029,M200,3,F33D,"[B956, M8627]",[],[],[]


# Match revised DtoD data with the BfS data from the database

In [4]:
cols_to_join = list(VALIDATION_COLS)
cols_to_join.remove(CASE_ID_COL)
cols_to_join.append(NORM_CASE_ID_COL)
# Ignore the patient ID in this dataset, because it is encrypted
cols_to_join.remove(PATIENT_ID_COL)

revised_cases, unmatched = revise(file_info, revised_cases_df, validation_cols=cols_to_join)
revised_cases

2022-11-21 08:56:38.197 | INFO     | src.service.bfs_cases_db_service:get_sociodemographics_for_hospital_year:95 - Read 6059 rows from the DB, for the hospital 'Hirslanden Klinik Beau-Site' in 2017


,aimedic_id,case_id,primary_diagnosis,secondary_diagnoses,primary_procedure,secondary_procedures,gender,age_years,age_days,gestation_age,duration_of_stay,ventilation_hours,grouper_admission_type,admission_date,admission_weight,grouper_discharge_type,discharge_date
0,118943,0041625890,C182,"[D630, C786, C778, Z926, R104, ...]",990410::20171228,[],M,56,0,0,4,0,01,2017-12-26,0,00,2017-12-30
1,119066,41256307,N390,"[R33, E1191, E039, F329, I2510, ...]",5794::20170119,[992211::20170119],W,77,0,0,5,0,01,2017-01-19,0,00,2017-01-24
2,119273,41314578,I211,"[I2511, E1190, E785, I1090, E050, ...]",006629::20170309,"[0040::20170309, 0045::20170309, 360799::20170309, 3722::20170309, 8853::20170309, ...]",W,80,0,0,10,0,01,2017-03-09,0,00,2017-03-19
3,119276,41433347,K565,"[K550, E876, F329, K910, E871]",5452::20170630,"[9607::20170628, 009A13::20170630, 4562::20170630, 469910::20170630, 890A32::20170630]",W,70,0,0,9,0,01,2017-06-28,0,00,2017-07-07
4,119313,41439665,I5001,"[I420, I2519, I489, N19, I513, ...]",887221::20170714,[],M,58,0,0,9,0,01,2017-07-05,0,06,2017-07-14
5,119564,0041419449,C251,"[D630, C787, Z926, E890, C7988, ...]",990410::20170621,[9915::20170616],W,67,0,0,8,0,01,2017-06-16,0,00,2017-06-24
6,120055,41384920,J14,"[I1100, I5014, E789, E559, A099, ...]",992213::20170515,[],M,80,0,0,10,0,01,2017-05-15,0,00,2017-05-25
7,120209,41301906,K625,"[I489, Z921, I350, Z953, I479, ...]",469922::20170228,"[4523::20170228, 451310::20170228]",W,81,0,0,5,0,01,2017-02-27,0,00,2017-03-04
8,120311,0041455542,C168,"[Z850, Z926, Z903, C787, C786, ...]",9435::20170720,[9438::20170720],M,74,0,0,8,0,01,2017-07-20,0,00,2017-07-28
9,120473,0041459894,C56,"[C786, Z926, K818, G572, Z9581, ...]",6862::20170727,"[544X12::20170727, 405211::20170727, 405929::20170727, 544X99::20170727, 6542:B:20170727, ...]",W,51,0,0,7,0,01,2017-07-27,0,00,2017-08-03


In [5]:
if unmatched.shape[0] > 0:
    display(unmatched)

In [6]:
revisions_update, diagnoses_update, procedures_update = group(revised_cases)
revisions_update[REVISION_DATE_COL] = '2022-12-31'

2022-11-21 08:56:38.721 | INFO     | src.revised_case_normalization.notebook_functions.group:group:86 - Grouping 47 cases ...
2022-11-21 08:56:39.826 | SUCCESS  | src.revised_case_normalization.notebook_functions.group:group:91 - Grouped 47 cases into: 47 revisions, 430 diagnoses rows, 195 procedure rows


In [7]:
revisions_update

,aimedic_id,drg,drg_cost_weight,effective_cost_weight,pccl,revision_date
0,118943,G60A,1.25,1.25,4,2022-12-31
1,119066,L63B,1.04,1.04,4,2022-12-31
2,119273,F52A,2.35,2.35,4,2022-12-31
3,119276,G18A,2.79,2.79,4,2022-12-31
4,119313,F62B,1.38,1.06,4,2022-12-31
5,119564,H61A,1.33,1.33,4,2022-12-31
6,120055,E77D,1.18,1.18,4,2022-12-31
7,120209,G46B,1.45,1.45,4,2022-12-31
8,120311,G60A,1.25,1.25,4,2022-12-31
9,120473,N01A,4.51,4.51,4,2022-12-31


In [8]:
update_db(revisions_update, diagnoses_update, procedures_update)

2022-11-21 08:56:39.848 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:259 - Trying to insert 47 cases into the 'Revisions' table ...
2022-11-21 08:56:41.526 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_revisions:296 - Inserted 47 cases into the 'Revisions' table
2022-11-21 08:56:41.529 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:309 - Trying to insert 430 rows into the 'Diagnoses' table ...
2022-11-21 08:57:02.426 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_diagnoses:347 - Inserted 430 rows into the 'Diagnoses' table
2022-11-21 08:57:02.432 | INFO     | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:358 - Trying to insert 195 rows into the 'Procedures' table ...
2022-11-21 08:57:04.650 | SUCCESS  | src.service.bfs_cases_db_service:insert_revised_cases_into_procedures:404 - Inserted 195 rows into the 'Procedures' table
